In [1]:
# Importar librerías
from scipy.io.wavfile import read
import numpy as np
from python_speech_features.base import mfcc
import os
import glob

In [ ]:
# Cambio de directorio
os.chdir('../data/raw')

In [17]:
os.chdir('../')

In [3]:
# Guardar lista de folders
folder_files = os.listdir()

# Variables por conveniencia para el for
num_ceps = 25
full_features = np.matrix([1] * (num_ceps + 2))
k = 0

In [ ]:
for folder in folder_files:
    os.chdir(folder)
    target = 1 if 'male' in folder else 0
    # Listar sólo archivos .wav
    audio_files = [audio for audio in glob.glob('*.wav')]
    for file in audio_files:
        # Leer archivo del folder i
        fs, audio = read(file)
        audio = audio[0:fs]

        # Vamos a obtener sus coeficientes.
        # Parámetros que recibe:
        #   - signal: la señal de audio.
        #   - winlen: tamaño de la ventana en segundos.
        #   - winstep: tamaño del avance (se utiliza para el prcentaje de muestras que habrá entre ventanas). En este caso se tiene un overlap de 50%.
        #   - numcep: número de coeficientes de Mel.        coeffs.shape
        #   - nfilt: número de filtros del banco.
        coeffs = mfcc(signal = audio, samplerate = fs, winlen = 0.030, winstep = 0.015, numcep = num_ceps, nfilt = 30, nfft = 1500)

        # Matriz de 1's (male) o 0's (female)
        classes = np.matrix([target]*coeffs.shape[0])
        print(classes.T.shape)

        index = np.matrix([k] * coeffs.shape[0])

        print(index.T.shape)

        # Combinar 'coeffs' y 'classes'
        features = np.hstack([coeffs, classes.T])
        print(features.shape)
        features = np.hstack([index.T, features])

        # Combinar features actuales con anteriores y redefinir fake_matrix
        full_features = np.vstack([full_features, features])
    os.chdir('../')

In [34]:
# Guardar full_features excepto los 1's de la primera fila
np.savetxt('../features/full_features.txt', full_features[1:, ], delimiter = ',', fmt = '%f')